In [104]:
# import needed packages

import pandas as pd
import datarobot as dr
import math
import matplotlib.pyplot as plt

# read in master data 
NGIC = pd.read_csv("/Users/quncai.zou/Documents/NGIC/input_data_data_robot2.csv", encoding = 'ISO-8859-1')
pd.options.display.max_columns = 112 # enable options to show all the columns
NGIC.head()

/Users/quncai.zou/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (48,49,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ndspolicyid,EndorsementNum,VehicleNum,TermEffectiveDate,TermExpirationDate,effectivedate,pol_year,term,policysuffixnum,SourceSystemCode,GoverningStateCode,OriginalEffectiveDate,tenure,num_non_drv,num_excl_drv,num_oth_ins_drv,num_drv,PPAVehNum,package,EFT,ADQ,SR22,PIF,Paperless,tier_rad6,BIGroup,VehAge_incept,HO,class_ins_exp,any_acc_ind_p,any_acc_ind_gt36_p,ActionCode_v5,SRMActionCode,ProductVersionName,DistributionChannel,veh_hist_v2,ModelYearNum,VIN,symbol8bi,symbol8mp,symbol8cp,symbol8cl,GaragingZipCode,Terr_bipd,Terr_CP,Terr_CL,Terr_mppip,limitbi,limitpd,dedcl,dedcp,PIP_lmt,limitmp,YoungDrv,OldDrv,Parent_Child,VehAge,youthful_pni,youthful_on_pol,parent_child_rel_on_pol,PNIAge,BI_EP,PD_EP,MED_EP,PIP_EP,COLL_EP,COMP_EP,biinc,pdinc,mpinc,pipinc,cpinc,clinc,biclc,pdclc,mpclc,pipclc,cpclc,clclc,biexp,pdexp,mpexp,pipexp,cpexp,clexp,tot_exp,tot_ep,totinc,tot_clc,SpeedsCount_gt36_pol,SppCount_gt36_pol,SpdCount_gt36_pol,DwisCount_gt36_pol,MinorCount_gt36_pol,MajorCount_gt36_pol,AfaCount_gt36_pol,NafaCount_gt36_pol,SpeedsCount_pol,SppCount_pol,SpdCount_pol,DwisCount_pol,MinorCount_pol,MajorCount_pol,AfaCount_pol,NafaCount_pol,biprem_theo,pdprem_theo,cpprem_theo,clprem_theo,mpprem_theo,pipprem_theo
0,1019432,0,1,01JAN2013:00:00:00,01JUL2013:00:00:00,01JAN2013:00:00:00,2013,6,0,N,OH,01JUL2008:00:00:00,54,0,0,0,2,3,0,0,0,0,1,0,3.0,4,12,1,z.a.N,NaN,NaN,E,F,RAD3.5,Direct,15.0,2001,1G8JU52F81Y546761,B,B,04,09,45014,18,01,09,11,100/300,50,500,500,NaN,NaN,22,58,1,12.0,0,0,1,58,56.564384,58.997260,0.0,0.0,69.336986,20.071233,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.304110,0.304110,NaN,NaN,0.304110,0.304110,0.304110,204.969863,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.09,34.01,3.72,21.82,NaN,NaN
1,1019432,0,2,01JAN2013:00:00:00,01JUL2013:00:00:00,01JAN2013:00:00:00,2013,6,0,N,OH,01JUL2008:00:00:00,54,0,0,0,2,3,0,0,0,0,1,0,3.0,4,13,1,z.a.N,NaN,NaN,E,F,RAD3.5,Direct,7.0,2000,1B7HC16YXYS510210,E,9,10,10,45014,18,01,09,11,100/300,50,500,500,NaN,NaN,22,58,1,13.0,0,0,1,58,63.254795,66.295890,0.0,0.0,66.295890,22.504110,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.304110,0.304110,NaN,NaN,0.304110,0.304110,0.304110,218.350685,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48.59,43.38,5.19,21.36,NaN,NaN
2,1019432,0,3,01JAN2013:00:00:00,01JUL2013:00:00:00,01JAN2013:00:00:00,2013,6,0,N,OH,01JUL2008:00:00:00,54,0,0,0,2,3,0,0,0,0,1,0,3.0,4,3,1,z.a.N,NaN,NaN,E,F,RAD3.5,Direct,4.0,2010,4A32B2FFXAE008943,G,J,17,33,45014,18,01,09,11,100/300,50,500,500,NaN,NaN,22,58,1,3.0,0,0,1,58,66.904110,70.553425,0.0,0.0,145.364384,26.153425,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.304110,0.304110,NaN,NaN,0.304110,0.304110,0.304110,308.975342,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52.23,46.63,10.24,55.46,NaN,NaN
3,1019432,1,2,01JAN2013:00:00:00,01JUL2013:00:00:00,22APR2013:00:00:00,2013,6,0,N,OH,01JUL2008:00:00:00,54,0,0,0,2,2,0,0,0,0,1,0,3.0,4,13,1,z.a.N,NaN,NaN,E,G,RAD3.5,Direct,7.0,2000,1B7HC16YXYS510210,E,9,10,10,45014,18,01,09,11,100/300,50,500,500,NaN,NaN,22,58,1,13.0,0,0,1,58,37.589041,39.506849,0.0,0.0,39.506849,14.191781,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.191781,0.191781,NaN,NaN,0.191781,0.191781,0.191781,130.794521,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.66,30.05,3.02,13.64,NaN,NaN
4,1019432,1,3,01JAN2013:00:00:00,01JUL2013:00:00:00,22APR2013:00:00:00,2013,6,0,N,OH,01JUL2008:00:00:00,54,0,0,0,2,2,0,0,0,0,1,0,3.0,4,3,1,z.a.N,NaN,NaN,E,G,RAD3.5,Direct,4.0,2010,4A32B2FFXAE008943,G,J,17,33,45014,18,01,09,11,100/300,50,500,500,NaN,NaN,22,58,1,3.0,0,0,1,58,39.890411,41.808219,0.0,0.0,85.917808,16.493151,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.191781,0.191781,NaN,NaN,0.191781,0.191781,0.191781,184.109589,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.18,32.30,5.97,35.41,NaN,NaN


In [ ]:
# NGIC.groupby(["GoverningStateCode","pol_year"])["clinc","clprem_theo"].agg('sum')
# NGIC.groupby(["GoverningStateCode"])["clinc","clprem_theo"].agg('sum')

In [127]:
# upload data to DataRobot and get ready for auto-pilot
def NGIC_Data_subset(projectname, feature_list, API_Token, EndPoint):
    dr.Client(token=API_Token, endpoint=EndPoint)
    project = dr.Project.create(sourcedata, project_name=projectname)
    featurelist = project.create_featurelist('feature_base', feature_list)
#     project.start_autopilot(featurelist.id
    return project

In [135]:
def loss_ratio_relativity(PROJECT_ID, MODEL_ID, current_pred, loss):
# get predictions for holdout set
    model = dr.Model.get(project=PROJECT_ID, model_id=MODEL_ID)
    # Calculate new training predictions on holdout partition of dataset
    training_predictions_job = model.request_training_predictions(dr.enums.DATA_SUBSET.HOLDOUT)
    training_predictions = training_predictions_job.get_result_when_complete()
    # Fetch training predictions as data frame
    preds = training_predictions.get_all_as_dataframe()
    print("done with predictions")
    
# merge downloaded predictions with original data to get additional features
    prediction_actual = pd.merge(preds,sourcedata,on="row_id",how="left")
    print(prediction_actual.head())
    return prediction_actual

In [139]:
# state: NY, Coverage: pd - need to change state, coverage, and theoretical premium every time
API_Token = 'NWRjNGZhNjMzNGE5ODcwZDQzZmZiN2E0OnpyZzV5STI5VEVxeTNwbU9JV1h5eHNMQzhLWHBUbURKSFE5MWVueFNGblk9'
EndPoint = 'https://app.datarobot.com/api/v2'

f_filter = 'GoverningStateCode == "CA" and clexp > 0 and clprem_theo > 0' # pls change "NY" and clprem_theo to corresponding state and coerage
projectname = 'NGIC_CA_CL_w_offset' # pls change project name to corresponding state/coverage
sourcedata = NGIC.query(f_filter)
sourcedata.loc[:,"offset"] = sourcedata.apply(lambda x: math.log(x.clprem_theo), axis = 1) #pls change clprem_theo to corresponding coverage

#########################################################################################################
feature_list = ['EndorsementNum', 'VehicleNum', 'pol_year', 'term', 'tenure',
               'num_non_drv', 'num_excl_drv', 'num_oth_ins_drv','num_drv','PPAVehNum',
               'SR22','tier_rad6','BIGroup','VehAge_incept','class_ins_exp',
               'any_acc_ind_p','any_acc_ind_gt36_p','DistributionChannel','veh_hist_v2',
               'symbol8bi','symbol8cl','symbol8cp','Terr_bipd','Terr_CL','Terr_CP','limitbi','limitpd',
               'dedcl','dedcp','YoungDrv','OldDrv','Parent_Child','VehAge',
               'youthful_pni','youthful_on_pol','parent_child_rel_on_pol','PNIAge',
                'SpeedsCount_gt36_pol','SppCount_gt36_pol','SpdCount_gt36_pol','DwisCount_gt36_pol',
                'MinorCount_gt36_pol','MajorCount_gt36_pol','AfaCount_gt36_pol','NafaCount_gt36_pol',
                'SpeedsCount_pol','SppCount_pol','SpdCount_pol','DwisCount_pol',
                'MinorCount_pol','MajorCount_pol','AfaCount_pol','NafaCount_pol']


#call the function NGIC_Data_subset to prepare for DR project
project = NGIC_Data_subset(projectname, feature_list, API_Token, EndPoint)
# advanced_options = dr.AdvancedOptions(offset=['offset'])
print("data is uploaded - go to DR and continue...")

data is uploaded - go to DR and continue...


In [150]:
# ## get predictions for holdout dataset

PROJECT_Id = '5e4c1923f7cf1c05f8893eae' #pls change to the appropriate project_id
MODEL_ID = '5e4c21a2b39ec966c0ff4d87' #pls change to the appropriate model_id
current_pred = "clprem_theo"          #pls change to the appropriate xxxx_theo
loss = "clinc"                        #pls change to the appropriate project_id

#######################
sourcedata["index"] = sourcedata.index
sourcedata["row_id"] = sourcedata["index"].rank()-1
print("check 2")

output_data = loss_ratio_relativity(PROJECT_Id, MODEL_ID, current_pred, loss)

print("downloading predictions from DataRobot is completed.")
    
output_data["ratio"] = output_data["prediction"]/output_data[current_pred]
output_data['bin'] = pd.qcut(output_data['ratio'], q=10, precision=0)

output_data.sort_values("ratio")
print(output_data.head(20))

LR_DR = output_data.groupby(["bin"])[loss].agg('sum')/output_data.groupby(["bin"])['prediction'].agg('sum')
LR_NGIC = output_data.groupby(["bin"])[loss].agg('sum')/output_data.groupby(["bin"])[current_pred].agg('sum')

print(LR_DR)

LR_DR = LR_DR/LR_DR[0]
LR_NGIC = LR_NGIC/LR_NGIC[0]
x_bin = [1,2,3,4,5,6,7,8,9,10]
data_dict = {"bin":x_bin, "LR_DR":LR_DR, "LR_NGIC":LR_NGIC}
data_4_plot = pd.DataFrame(data_dict)

plt.plot(data_4_plot["bin"],data_4_plot["LR_DR"], label = "LR_DR")
plt.plot(data_4_plot["bin"],data_4_plot["LR_NGIC"], label = "LR_NGIC")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
plt.show()